In [1]:
from scamp import *

Testing for working audio driver...
jackdmp 1.9.22
Copyright 2001-2005 Paul Davis and others.
Copyright 2004-2016 Grame.
Copyright 2016-2023 Filipe Coelho.
jackdmp comes with ABSOLUTELY NO WARRANTY
This is free software, and you are welcome to redistribute it
under certain conditions; see the file COPYING for details
JACK server starting in realtime mode with priority 10
self-connect-mode is "Don't restrict self connect requests"
creating alsa driver ... hw:0|hw:0|1024|2|48000|0|0|nomon|swmeter|-|32bit


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5703:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default
fluidsynth: error: Failed to open the "default" audio device
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'coreaudio'.
fluidsynth: This build of fluidsynth supports the follo

In [3]:
session = Session()
session.fast_forward_in_beats(200)
# piano1 = session.new_part("Piano")
supercollider = session.new_osc_part("organReed", 57120, "192.168.1.151")

pitches = [55, 66, 33, 75]

session.start_transcribing()

for pitch in pitches:
    supercollider.play_note(pitch, 1.0, 0.25)

session.stop_transcribing()

Performance([
   PerformancePart(name='organReed', instrument_id=('organReed', 0), voices={
      '_unspecified_': [
         PerformanceNote(start_beat=0.0, length=0.25, pitch=55, volume=1.0, properties=NoteProperties()),
         PerformanceNote(start_beat=0.25, length=0.25, pitch=66, volume=1.0, properties=NoteProperties()),
         PerformanceNote(start_beat=0.5, length=0.25, pitch=33, volume=1.0, properties=NoteProperties()),
         PerformanceNote(start_beat=0.75, length=0.25, pitch=75, volume=1.0, properties=NoteProperties())
      ]
   })
])